In [1]:
from sys import path
path.append('..//modules')
path.append('..//dumps//reg_dmo_pdf')
import time

import psycopg2 as pg
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import create_engine, MetaData, Table
import pandas.io.sql as psql

#create a postgres engine and a pg connection
from modules.db_connector import *
print("DB connecting engine imported")

DB connecting engine imported


In [4]:
#import Sal(2023) module for calling WFS to gpdf
from modules import call_wfs
print("Module call_wfs imported")

start = time.time()
print("Making call to WFS")
#extract layer from WFS and transform to a gpdf containing all SK municipalities (it takes about 2 minutes)
gpdf_gku = wfs2gp_df('WFS_zbgis_administrativne_hranice_wfs:obec', 'https://zbgisws.skgeodesy.sk/zbgis_administrativne_hranice_wfs/service.svc/get?')
print("gpdf_gku retrieved in : ", round(time.time() - start, 2), " seconds.")


#load the created gpdf in to postgis db
gpdf_gku.to_postgis('sk_municipalities_gku', engine, schema = 'gtlab', if_exists='replace', index=False, 
                   dtype={'geometry': Geometry(geometry_type='MultiPolygon', srid= '5514')})
#print("gpdf_gku loaded in db under name sk_municipalities_gku in: ", round(time.time() - start, 2), " seconds.")

Module call_wfs imported
Making call to WFS
2927
gpdf_gku retrieved in :  107.77  seconds.


In [5]:
gpdf_gku

,geometry,gml_id,OBJECTID,Dátum_aktualizácie_objektu_,Presnosť_horizontálna,Aktuálny_stav_objektu,DIGEST_kód,Číslo_obce,Názov_obce,Číslo_okresu,...,Kód_štátu,Oblasť,Kód_oblasti,Kraj,Kód_kraja,Okres,Kód_okresu,Obec,Kód_obce,GLOBALID
0,MULTIPOLYGON Z (((-192091.390 -1236228.940 0.0...,obec.17921,17921,2024-01-25T00:00:00,1,9,FA004,522473,Horňa,809,...,SK0,Východné Slovensko,SK04,Košický kraj,SK042,Sobrance,SK0429,Horňa,SK0429522473,{00F1B361-714D-4F0D-A98C-CB9708B895A6}
1,MULTIPOLYGON Z (((-522938.460 -1321491.630 0.0...,obec.17922,17922,2024-01-25T00:00:00,1,9,FA004,501361,Tôň,401,...,SK0,Západné Slovensko,SK02,Nitriansky kraj,SK023,Komárno,SK0231,Tôň,SK0231501361,{0107485E-B8FD-4C26-8443-2C2D5A275FED}
2,MULTIPOLYGON Z (((-487987.470 -1189011.630 0.0...,obec.17923,17923,2024-01-25T00:00:00,1,9,FA004,557412,Sedmerovec,302,...,SK0,Západné Slovensko,SK02,Trenčiansky kraj,SK022,Ilava,SK0222,Sedmerovec,SK0222557412,{0151B30F-DACE-44F0-94E1-08218B422631}
3,MULTIPOLYGON Z (((-484814.210 -1259307.230 0.0...,obec.17924,17924,2024-01-25T00:00:00,1,9,FA004,500437,Ladice,407,...,SK0,Západné Slovensko,SK02,Nitriansky kraj,SK023,Zlaté Moravce,SK0237,Ladice,SK0237500437,{017B9DB0-51A1-487A-BFB8-5466B6F19204}
4,MULTIPOLYGON Z (((-215304.000 -1178137.700 0.0...,obec.17925,17925,2024-01-25T00:00:00,1,9,FA004,559610,Rokytovce,705,...,SK0,Východné Slovensko,SK04,Prešovský kraj,SK041,Medzilaborce,SK0415,Rokytovce,SK0415559610,{01A99DE5-31A4-4DE0-9208-C8EAE6213967}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2922,MULTIPOLYGON Z (((-490388.610 -1188871.360 0.0...,obec.20843,20843,2024-01-25T00:00:00,1,9,FA004,512885,Bolešov,302,...,SK0,Západné Slovensko,SK02,Trenčiansky kraj,SK022,Ilava,SK0222,Bolešov,SK0222512885,{FE6A200E-D115-4CBE-BB25-6CFE6BE8731D}
2923,MULTIPOLYGON Z (((-185958.390 -1218130.930 0.0...,obec.20844,20844,2024-01-25T00:00:00,1,9,FA004,520438,Ladomirov,709,...,SK0,Východné Slovensko,SK04,Prešovský kraj,SK041,Snina,SK0419,Ladomirov,SK0419520438,{FF228B80-B0C0-4C08-A0A6-A89B4EF8F8E5}
2924,MULTIPOLYGON Z (((-224987.930 -1195900.270 0.0...,obec.20845,20845,2024-01-25T00:00:00,1,9,FA004,529061,Rohožník,702,...,SK0,Východné Slovensko,SK04,Prešovský kraj,SK041,Humenné,SK0412,Rohožník,SK0412529061,{FF2A04A2-10C3-41BF-BA3D-686E4AAD4E30}
2925,MULTIPOLYGON Z (((-394987.050 -1167201.400 0.0...,obec.20846,20846,2024-01-25T00:00:00,1,9,FA004,509671,Horná Lehota,503,...,SK0,Stredné Slovensko,SK03,Žilinský kraj,SK031,Dolný Kubín,SK0313,Horná Lehota,SK0313509671,{FFDB63D7-2E22-4659-9529-F738831B43D9}


In [3]:
start = time.time()
#import module for scrapping MINDOPs pdf registry of DMOs, further extraction of base data on dmos and transforming the retrieved information into a df. 
from modules import get_dmo_base_data
print("Module for scrapping DMO registry imported")
#invoke to module to create the desired df
df_dmo = get_dmo_base_data.get_dmo_base()
print("df_dmo created")
df_dmo.to_sql("sk_dmos_raw", engine, schema='gtlab', if_exists='replace')
print("df_dmo loaded in db under name sk_dmos_raw in: ", round(time.time() - start, 2), " seconds.")

Module for scrapping DMO registry imported
0 OOCR Región Vysoké Tatry  0_OOCR_Región_Vysoké_Tatry.pdf
1 OOCR SEVERNÝ SPIŠ - PIENINY  1_OOCR_SEVERNÝ_SPIŠ_-_PIENINY.pdf
2 OOCR Visit Košice   2_OOCR_Visit_Košice.pdf
3 OOCR SLNEČNÝ HONT  3_OOCR_SLNEČNÝ_HONT.pdf
4 OOCR REGIÓN HOREHRONIE  4_OOCR_REGIÓN_HOREHRONIE.pdf
5 OOCR „ŠARIŠ“ – BARDEJOV  5_OOCR_„ŠARIŠ“_–_BARDEJOV.pdf
6 OOCR Región Gron  6_OOCR_Región_Gron.pdf
7 OOCR Trenčianske Teplice  7_OOCR_Trenčianske_Teplice.pdf
8 OOCR Región Horné Považie  8_OOCR_Región_Horné_Považie.pdf
9 OOCR Trnava Tourism  9_OOCR_Trnava_Tourism.pdf
10 OOCR Rajecká dolina  10_OOCR_Rajecká_dolina.pdf
11 OOCR Horný Zemplín  11_OOCR_Horný_Zemplín.pdf
12 OOCR Turiec  12_OOCR_Turiec.pdf
13 Nitrianska OCR  13_Nitrianska_OCR.pdf
14 Zemplínska OOCR  14_Zemplínska_OOCR.pdf
15 OOCR Stredné Slovensko  15_OOCR_Stredné_Slovensko.pdf
16 OOCR Malá Fatra  16_OOCR_Malá_Fatra.pdf
17 OOCR Región Banská Štiavnica  17_OOCR_Región_Banská_Štiavnica.pdf
18 OOCR Vysoké Tatry - Podhori

In [4]:
start = time.time()
#import module for extracting base dmo members info (name, identification of attribute municipaltity/city/capital) from the dumped pdfs 
#and transform the information into a structured df
from modules.get_dmo_base_members import *
print("Module for extracting DMO members to df imported")
#invoke the module with df_dmo created in previous step 
df_members = get_dmo_members(df_dmo)
#load the created df to db
df_members.to_sql("sk_dmo_members_raw", engine, schema='gtlab', if_exists='replace')
print("df_members loaded in db under name sk_dmo_members_raw in: ", round(time.time() - start, 2), " seconds.")

foo
Module for extracting DMO members to df imported
0 6
1 16
2 68
3 17
4 22
5 5
6 48
7 9
8 24
9 28
10 23
11 28
12 30
13 22
14 26
15 43
16 27
17 71
18 14
19 25
20 29
21 20
22 84
23 33
24 24
25 64
26 14
27 60
28 17
29 9
30 27
31 39
32 53
33 28
34 28
35 22
36 66
37 4
38 37
0       vysoké tatry
1             poprad
2              štrba
3               None
4               None
            ...     
1205            None
1206            None
1207            None
1208            None
1209            None
Name: municipality_name, Length: 1210, dtype: object
df_members loaded in db under name sk_dmo_members_raw in:  14.55  seconds.


In [6]:
start = time.time()
#import module for basic clean up of municipality names
from modules.clean_dmo_municipality_name import clean_municipality_names
print("Module for cleaning municipality names imported")
#invoke module
df_dmo_boundaries = clean_municipality_names(df_members.query('lau2 ==1'))
#load the transformed data in to db
df_dmo_boundaries.to_sql("sk_dmo_boundaries_raw", engine, schema='gtlab', if_exists='replace')
print("df_dmo_boundaries loaded in db under name sk_dmo_boundaries_raw (WARNING df contains duplicities use module in next step) in: ", round(time.time() - start, 2), " seconds.")

Module for cleaning municipality names imported
0 vysoké tatry
1 poprad
2 štrba
6 stará ľubovňa
7 chmeľnica
8 nová ľubovňa
9 hniezdne
10 vyšné ružbachy
11 lechnica
12 lesnica
13 litmanová
22 košice
90 dudince
107 brezno
108 mýto pod ďumbierom
109 čierny balog
110 bystrá
111 horná lehota
112 predajná
113 lom nad rimavicou
114 šumiac
115 brusno
116 jasenie
117 nemecká
118 telgárt
119 hronec
120 osrblie
121 valaská
122 polomka
123 heľpa
129 bardejov
130 svidník
134 žarnovica
135 nová baňa
136 horné hámre
137 župkov
138 ostrý grúň
139 repište
140 kľak
141 tekovská breznica
142 malá lehota
143 veľká lehota
144 voznica
145 hronský beňadik
146 brehy
147 píla
148 rudno nad hronom
149 bzenica
150 hrabičov
151 žiar nad hronom
152 sklené teplice
153 hliník nad hronom
182 trenčianske teplice
191 považská bystrica
192 nimnica
193 papradno
194 horná mariková
195 plevník-drienové
196 pružina
197 vrchteplá
198 záskalie
215 trnava
216 jaslovské bohunice
217 ružindol
218 smolenice
219 suchá nad parnou
2

In [6]:
conn_1 = pg.connect("dbname=dbis user=postgres host=localhost port=5432 password=17021987")
conn_1.close()
#FROM THIS CHECK POINT WE'RE working only with data in the DB
df_dmo_muns = psql.read_sql("SELECT * FROM gtlab.sk_dmo_boundaries_raw order by dmo_id, member_id", conn_1)
df_gku_muns = psql.read_sql('SELECT "Názov_obce", "Kraj", "Kód_obce"  FROM gtlab.sk_municipalities_gku', conn_1)
#import the module from identifying duplicate muns in dmo
from modules import drop_duplicate_municipalities
#invoke the module for creating a df with unique municipalities in a DMO  and relevant ID for joining with the spatial layer of SK municipalities 
df_dmo_muns_w_susr_codes = drop_duplicate_municipalities.drop_duplicates(df_gku_muns, df_dmo_muns)
#load the transformed df to the DB
df_dmo_muns_w_susr_codes.to_sql("sk_dmo_boundaries_susr_codes", engine, schema='gtlab', if_exists='replace')
print("df_dmo_muns_w_susr_codes loaded in db under name df_dmo_muns_w_susr_codes (USE THIS TO JOIN WITH sk_municipalities_gku) in: ", round(time.time() - start, 2), " seconds.")


#GKU uses SRID 5514 

#FOR PERSONAL REASONS I'M LEAVING A THE INTERMEDIATE RESULTS IN THE DB (NO HORDING, ONE NEVER KNOWS...)
#The final step is to assign appropriate primary keys and constraints between all tables. Since I'm chasing a deadline I'm leaving the SQL queries here:
"""
ALTER TABLE gtlab.sk_dmos_municipalties_w_gku RENAME COLUMN geometry  TO geom;
ALTER TABLE gtlab.sk_municipalities_gku RENAME COLUMN geometry  TO geom;

SELECT UpdateGeometrySRID('gtlab','sk_dmos_municipalties_w_gku', 'geom', 5514)
SELECT UpdateGeometrySRID('gtlab','sk_municipalities_gku', 'geom', 5514)

ALTER TABLE gtlab.sk_dmo_members_raw ADD COLUMN dmo_member_id text;
UPDATE gtlab.sk_dmo_members_raw SET dmo_member_id = dmo_id::text || '_' || member_id || '_' || member_name;
ALTER TABLE gtlab.sk_dmo_boundaries_susr_codes ADD COLUMN dmo_member_id text;
UPDATE gtlab.sk_dmo_boundaries_susr_codes SET dmo_member_id = dmo_id::text || '_' || member_id || '_' || member_name_x;
ALTER TABLE gtlab.sk_dmo_members_raw ADD PRIMARY KEY (dmo_member_id); 
ALTER TABLE gtlab.sk_dmo_boundaries_susr_codes ADD PRIMARY KEY ("Kód_obce"); 
ALTER TABLE gtlab.sk_dmos_raw ADD PRIMARY KEY (dmo_id);
ALTER TABLE gtlab.sk_municipalities_gku ADD PRIMARY KEY ("Kód_obce");


ALTER TABLE gtlab.sk_dmo_boundaries_susr_codes
	ADD CONSTRAINT fk_sk_dmo_boundaries_susr_codes_with_sk_dmo_members  FOREIGN KEY (dmo_member_id) REFERENCES gtlab.sk_dmo_members_raw (dmo_member_id);

ALTER TABLE gtlab.sk_dmo_members_raw 
	ADD CONSTRAINT fk_sk_dmo_members_with_sk_dmo  FOREIGN KEY (dmo_id) REFERENCES gtlab.sk_dmos_raw (dmo_id) ;

ALTER TABLE gtlab.sk_dmo_boundaries_susr_codes
	ADD CONSTRAINT fk_sk_dmo_boundaries_susr_codes_with_sk_dmo  FOREIGN KEY (dmo_id) REFERENCES gtlab.sk_dmos_raw (dmo_id) ;

ALTER TABLE gtlab.sk_dmo_boundaries_susr_codes
	ADD CONSTRAINT fk_sk_dmo_boundaries_susr_codes_with_sk_municipalities_gku  FOREIGN KEY ("Kód_obce") REFERENCES gtlab.sk_municipalities_gku ("Kód_obce");
"""

#AFTER THIS JUST RUN THE QUERY OVER DB AND YOU GOT THE FINAL TABLE
"""
CREATE TEMP TABLE gtlab.sk_dmos_municipalities_w_gku AS
SELECT * FROM gtlab.sk_dmo_boundaries_susr_codes
LEFT JOIN gtlab.sk_municipalities_gku USING ("Kód_obce")
"""

C:\Users\csb\AppData\Local\Temp\ipykernel_22864\222367627.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dmo_muns = psql.read_sql("SELECT * FROM gtlab.sk_dmo_boundaries_raw order by dmo_id, member_id", conn_1)
C:\Users\csb\AppData\Local\Temp\ipykernel_22864\222367627.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_gku_muns = psql.read_sql('SELECT "Názov_obce", "Kraj", "Kód_obce"  FROM gtlab.sk_municipalities_gku', conn_1)


0 0 1 vysoké tatry False
1 0 2 poprad False
2 0 3 štrba False
3 1 1 stará ľubovňa False
4 1 2 chmeľnica False
5 1 3 nová ľubovňa False
6 1 4 hniezdne False
7 1 5 vyšné ružbachy False
8 1 6 lechnica False
9 1 7 lesnica False
10 1 8 litmanová False
11 2 1 košice True
12 2 1 košice True
13 2 1 košice True
14 2 1 košice True
15 2 1 košice True
16 2 1 košice True
17 2 1 košice True
18 2 1 košice True
19 2 1 košice True
20 2 1 košice True
21 2 1 košice True
22 2 1 košice True
23 2 1 košice True
24 2 1 košice True
25 2 1 košice True
26 2 1 košice True
27 2 1 košice True
28 2 1 košice True
29 2 1 košice True
30 2 1 košice True
31 2 1 košice True
32 2 1 košice True
33 3 1 dudince False
34 4 1 brezno False
35 4 10 jasenie False
36 4 11 nemecká False
37 4 12 telgárt False
38 4 13 hronec False
39 4 14 osrblie False
40 4 15 valaská False
41 4 16 polomka False
42 4 17 heľpa False
43 4 2 mýto pod ďumbierom False
44 4 3 čierny balog False
45 4 4 bystrá True
4 4 bystrá True
46 4 4 bystrá True
47 4 5 ho